# PerfCounters Usage


This notebook showcases how to use the [PerfCounters package](https://github.com/ebursztein/perfcounters).

PerfCounters offers to classes depending on what you need to measure:
- `TimeCounters()` are used if you want to track timings
- `ValueCounters()` are used if you want to track values.

You can use both at the same time and they have a fairly similar API.

## Time measurements

### Basic usage
Here is a short example that demonstrate how to track timing by comparing 
the time it takes to use `random.random` vs `random.randint`

In [1]:
import random
from perfcounters import TimeCounters

cnts = TimeCounters()  # init the counter collection.

cnts.start('random')  # start a timing counter called random.
for x in range(100000):
    int(random.random())
cnts.stop('random')  # stop the random counter

cnts.start('randint')  # start a timing counter called randint.
for x in range(1000000):
    random.randint(0, 1)
cnts.stop('randint')  # stop the randint counter.

cnts.report(rounding=5) # report print all counter values in nicely formated tables.

random_val = cnts.get('random', format='ms')   # get the value of the counter in ms
print(f'random value: {random_val}')

╭─────────┬────────────╮
│ Name    │   Time (s) │
├─────────┼────────────┤
│ random  │    0.00583 │
│ randint │    0.25348 │
╰─────────┴────────────╯
random value: 5.83


You can also export the results in various formats including json (`to_json()`)
), Markdown (`to_md()`), HTML (`to_html()`). For example to export the counters serialized in json you can simply use  the `to_json()` function as follow: 

In [2]:
cnts.to_json(rounding=3)

'{"random": 0.006, "randint": 0.253}'

### Measuring laps
Timing counters can also be used to track how long each iteration of a loop is taking via `lap()`. Here is a short example

In [3]:
import time
from perfcounters import TimeCounters
cnts = TimeCounters()  # declaring our counters

cnts.start('random loop') # create counter
for _ in range(3):
    time.sleep(round(random.random(), 2))
    cnts.lap('random loop') # record lap time
cnts.stop('random loop')
cnts.stop_all()

cnts.report()  # we don't need to stop the counter. Report do it
laps = cnts.get_laps('random loop')  # get the laps values as a list
print(f'laps: {laps}')

# similar to counters you can pretty print laps
cnts.report_laps('random loop')

# and export them to various format
lap_json = cnts.laps_to_json('random loop')
print(f'laps json {lap_json}')


╭─────────────┬────────────╮
│ Name        │   Time (s) │
├─────────────┼────────────┤
│ random loop │       1.41 │
╰─────────────┴────────────╯
laps: [0.68, 0.38, 0.35, -9.989738464355469e-05]
╭───────┬──────────────╮
│   Lap │        Value │
├───────┼──────────────┤
│     0 │  0.68        │
│     1 │  0.38        │
│     2 │  0.35        │
│     3 │ -9.98974e-05 │
╰───────┴──────────────╯
laps json {"0": 0.68, "1": 0.38, "2": 0.35, "3": -9.989738464355469e-05}


As visible in the output above when outputing/returning laps counters `PerfCounters` do report the value of each lap, the cumulative time and statistics about the laps.

## Values Tracking
Value counters are used to track values. They are either directly set to
a given value with the `set()` method or can be incremented 
with the `inc()` method or decremented with `dec()` 

In [5]:
from perfcounters import ValueCounters
cnts = ValueCounters()
cnts.set('mycounter', 39)  # set counter value to 39
cnts.inc('mycounter', 3)  # increment counter by 3
val = cnts.get('mycounter') #  get the value of the counter
print('mycounter value:', val)

mycounter value: 42
